In [ ]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
load_dotenv()

In [ ]:
API_KEY = os.getenv("API_KEY")
API_KEY
print(API_KEY)

In [30]:
youtube = build("youtube", "v3", developerKey=API_KEY)

In [31]:
def search_result(query):
    """
    Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.search.html
    """
    request = youtube.search().list(
        part="snippet",
        q=query,
        maxResults=10,
    )

    return request.execute()

In [ ]:
def channel_stats(channelID):
    """
    Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.channels.html
    """
    request = youtube.channels().list(
        part="statistics",
        id=channelID
    )
    return request.execute()

response = channel_stats("UC50eaYi1tmwfHmYCeFwHbEA")
search = search_result("SoGreenItsLime")
print(response)
print(search)

In [ ]:
def get_video_ids(channelId):
    """
    Refer to the documentation: https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.search.html
    """
    videoIds = []
 
    request = youtube.search().list(
        part="snippet",
        channelId=channelId,
        type="video",
        maxResults=50,
        videoDuration="any",
        order="viewCount"
    )

    return request.execute()

results = get_video_ids("UC50eaYi1tmwfHmYCeFwHbEA")
print(results)
    # response = request.execute()
    # responseItems = response['items']

    # videoIds.extend([item['id']['videoId'] for item in responseItems if item['id'].get('videoId', None) != None])

    # if there is nextPageToken, then keep calling the API
    # while response.get('nextPageToken', None):
    #     request = youtube.search().list(
    #         part="snippet",
    #         channelId=channelId,
    #     )
    #     response = request.execute()
    #     responseItems = response['items']

    #     videoIds.extend([item['id']['videoId'] for item in responseItems if item['id'].get('videoId', None) != None])
    
    # print(f"Finished fetching videoIds for {channelId}. {len(videoIds)} videos found.")

    # return videoIds

In [ ]:
if __name__ == '__main__':
    # to get a list of IDs from a channel
    ids = get_video_ids("UC50eaYi1tmwfHmYCeFwHbEA")

    # to get results from search
    # response = search_result("pyscript")

    # get channel stats
    # response = channel_stats(channelID='UCzIxc8Vg53_ewaRIk3shBug')

    # get comments
    #response = comment_threads(videoID='Qo8dXyKXyME', to_csv=True)

    print(ids)